In [54]:
%%time
import sys
sys.path.insert(0, r'G:\PythonProjects\WineRecognition2')
import pickle
import os
import pandas as pd
from data_master import DataGenerator, DataLoader
import features
from mlflow_utils import log_mlflow_on_test

Wall time: 0 ns


In [52]:
%%time
MODEL_NAME = r"G:\PythonProjects\WineRecognition2\artifacts\train\CRF_26082021_132615"
DICTIONARY_PATH = r"G:\PythonProjects\WineRecognition2\data\dictionaries\Dict-byword_Halliday_Winesearcher_Wine_AU-only_completed_rows"
TEST_DATASET_PATH = r"G:\PythonProjects\WineRecognition2\data\text\WineSearcher_Wine_AU-all_rows-all_keys.txt"
OUTPUT_DIR = ''
START_TIME = ''
RUN_NAME = ''
COMPUTE_METRICS = False

Wall time: 0 ns


In [48]:
%%time
with open(os.path.join(MODEL_NAME, 'model', 'model.pkl'), 'rb') as file:
    model = pickle.load(file)

with open(TEST_DATASET_PATH) as file:
    sents = DataGenerator.generate_sents(
        file.read().split('\n')
    )

Wall time: 670 ms


In [49]:
%%time
freq_dict = DataLoader.load_frequency_dictionary(DICTIONARY_PATH)

Wall time: 43 ms


In [50]:
%%time
X_test = [features.sent2features(s, freq_dict) for s in sents]
y_test = [features.sent2labels(s) for s in sents]

feature_list = list(X_test[0][1].keys())

Wall time: 1.11 s


In [51]:
%%time
y_pred = model.predict(X_test)

Wall time: 1.8 s


In [55]:
%%time
if COMPUTE_METRICS:
    
    test_eval = sents.copy()
    for i, wine in enumerate(test_eval):
        for j, word in enumerate(wine):
            test_eval[i][j] += (y_pred[i][j],)
            
else:
    
    test_eval = []
    for i, wine in enumerate(sents):

        dct = dict.fromkeys(model.classes_, '')

        for j, (word, true_label) in enumerate(wine):

            if y_pred[i][j] in dct.keys():
                dct[y_pred[i][j]] += f'{word} '

        test_eval.append({key: value.rstrip() for key, value in dct.items()})

    test_eval = pd.DataFrame({key: [wine.get(key) for wine in test_eval] for key in model.classes_})

Wall time: 693 ms


In [38]:
%%time
run_params = {
    'model_name': MODEL_NAME,
    'dictionary_path': DICTIONARY_PATH,
    'dataset_path': TEST_DATASET_PATH,
    'output_dir': OUTPUT_DIR,
    'compute_metrics': COMPUTE_METRICS,
    'start_time': START_TIME,
    'runname': RUN_NAME,
    'features': feature_list
}

Wall time: 0 ns


In [42]:
log_mlflow_on_test(
    run_params=run_params,
    model=model,
    y_true=y_test,
    y_pred=y_pred,
    test_eval=test_eval
)